In [82]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt 


In [83]:
data = pd.read_excel("D:/Python Projects/Running Optimization/archive/Strava Running Data.xlsx")


In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Sr. no.               105 non-null    int64  
 1   start_date_local      105 non-null    object 
 2   type                  105 non-null    object 
 3   distance              105 non-null    float64
 4   moving_time           105 non-null    int64  
 5   elapsed_time          105 non-null    int64  
 6   total_elevation_gain  105 non-null    float64
 7   start_latlng          105 non-null    object 
 8   end_latlng            105 non-null    object 
 9   sport_type            105 non-null    object 
 10  start_date            105 non-null    object 
 11  timezone              105 non-null    object 
 12  achievement_count     105 non-null    int64  
 13  kudos_count           105 non-null    int64  
 14  comment_count         105 non-null    int64  
 15  athlete_count         1

In [85]:
data

,Sr. no.,start_date_local,type,distance,moving_time,elapsed_time,total_elevation_gain,start_latlng,end_latlng,sport_type,start_date,timezone,achievement_count,kudos_count,comment_count,athlete_count,photo_count,average_speed,max_speed
0,1,2023-12-09T09:09:19Z,Run,10879.7,4023,4617,91.4,"[40.70327935740352, -73.99619171395898]","[40.69340907968581, -73.97922154515982]",Run,2023-12-09T14:09:19Z,(GMT-05:00) America/New_York,4,4,0,3,0,2.704,6.228
1,2,2023-12-07T17:31:50Z,Run,1304.4,722,62993,0.0,"[40.7220459356904, -74.03641730546951]","[40.719722136855125, -74.03257705271244]",Run,2023-12-07T22:31:50Z,(GMT-05:00) America/New_York,0,0,0,1,0,1.807,4.044
2,3,2023-12-03T09:18:13Z,Run,17503.0,7370,7462,68.4,"[40.74991073459387, -73.98751585744321]","[40.73538766242564, -73.9793517999351]",Run,2023-12-03T14:18:13Z,(GMT-05:00) America/New_York,2,5,0,4,0,2.375,6.778
3,4,2023-12-02T09:41:14Z,Run,3457.8,1791,2170,3.9,"[40.72203076444566, -74.03641068376601]","[40.71398631669581, -74.03902692720294]",Run,2023-12-02T14:41:14Z,(GMT-05:00) America/New_York,0,2,0,3,0,1.931,4.522
4,5,2023-12-01T17:06:05Z,Run,10108.2,4128,4221,6.3,"[40.72190687991679, -74.03624514117837]","[40.71669920347631, -74.03264125809073]",Run,2023-12-01T22:06:05Z,(GMT-05:00) America/New_York,4,5,0,1,0,2.449,5.462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,101,2022-06-12T09:20:44Z,Run,7857.2,3900,4113,116.6,"[39.27252494730055, -76.69631998054683]","[39.27245143800974, -76.69639013707638]",Run,2022-06-12T13:20:44Z,(GMT-05:00) America/New_York,6,0,0,1,0,2.015,3.326
101,102,2022-05-28T11:11:38Z,Run,6067.2,3334,3555,88.2,"[39.272494772449136, -76.69640866108239]","[39.27246367558837, -76.69636884704232]",Run,2022-05-28T15:11:38Z,(GMT-05:00) America/New_York,3,0,0,1,0,1.820,3.172
102,103,2022-05-22T07:13:30Z,Run,4587.6,2263,2478,56.6,"[39.272621758282185, -76.69633297249675]","[39.27250734530389, -76.69635468162596]",Run,2022-05-22T11:13:30Z,(GMT-05:00) America/New_York,5,0,0,1,0,2.027,3.557
103,104,2022-05-21T07:27:40Z,Run,4313.6,2182,2426,55.0,"[39.27257808856666, -76.69626507908106]","[39.27250818349421, -76.69635669328272]",Run,2022-05-21T11:27:40Z,(GMT-05:00) America/New_York,4,0,0,1,0,1.977,3.874


In [86]:
features = data[['distance', 'total_elevation_gain', 'average_speed']]
target = data['moving_time']

In [87]:
scalar  = StandardScaler()
features = scalar.fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [88]:
# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Ensure target is of shape (n_samples, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


ValueError: could not determine the shape of object type 'Series'